In [1]:
using CSV
using DataFrames

# Read CSV files
df = CSV.read("student_response.csv", DataFrame)
class_data = CSV.read("class_data_corrected.csv", DataFrame)
student_preferences = CSV.read("student_preferences_corrected.csv", DataFrame)

# Extract specific columns from the DataFrame
name_vals = df.Name
a_like = collect(df.attendance_likelihood)
head_volunteer = df.head_volunteer

# Drop unwanted columns
drop_cols = [
    :Name, :attendance_likelihood, :head_volunteer,
    :stay_all_day, :event, :specific_event, :event_1,
    :specific_event_1, Symbol("Which do you prefer?    =====================>"),
    :Event_grader, :Proctor, :Runner_Floater, :photographer
]

# Drop columns and remove the last column
select!(df, Not(drop_cols))

# Display remaining columns
println(name_vals)

# Create dictionaries for mappings
ind_to_event = Dict{Int, String}()
event_to_ind = Dict{String, Int}()

ind_to_name = Dict{Int, String}()
name_to_ind = Dict{String, Int}()

# Map indices to names
for (i, name) in enumerate(name_vals)
    ind_to_name[i] = name
    name_to_ind[name] = i
end

# Map indices to events
for (i, event) in enumerate(class_data.Event)
    ind_to_event[i] = event
    event_to_ind[event] = i
end

# Create student preferences dictionary
s_pref = Dict{Int, Vector{Int}}()

cols = string.(names(student_preferences)[2:end])  # Skip the "Name" column

for row in eachrow(student_preferences)
    name = row.Name
    name_ind = name_to_ind[name]
    # Get indices of preferred classes and map them to event indices
    class_inds = string.(findall(!iszero, row[2:end]))
    class_names = [event_to_ind[ind] for ind in class_inds]

    s_pref[name_ind] = class_names
end

# Print mappings (for debugging if needed)
println(ind_to_name)
println(ind_to_event)
println(s_pref)


String31["Ivan Ge", "Daniel Ochoa", "Alan Choi", "Jacob Mazumdar", "Sophie Wang", "Matthew Barnett", "Robin Xiong", "Anna Kaganov", "Jack Liu", "Lance Berkey", "Sabrina Meng", "Riddhi Bhagwat", "Amie Kitjasateanphun", "Isabella Duan", "Charlotte Myers", "Hector Flores", "April Wu", "Kennedy Gore", "Rafaa Qanash", "Maxi Attiogbe", "Davut Muhammetgulyyev", "Anna Liu", "Andrea Cosio", "Robin", "Abayo Joseph Desire", "Eric Delgado", "Sophia Lee", "Serena Pei", "Reina Wang", "Lama Alahdal ", "David Robichaud", "Triston Mohamed", "Allison Park", "Christopher Wang", "Jean Yu", "Macy Hogsett", "Matthew Nunez", "Audrey Lim", "Alana Yang", "Nicole Pardal", "Hanson He", "Claire Chen", "Richard Chen", "Pari Rajesh", "Felicity Zhou", "Katherine Li", "Wilson Zhu", "Sukrith Velmineti", "Alvin Zheng", "Caleb Zhao", "Vicky Yan", "Pragnya Govindu", "Kevin Toledo", "Ann Hashimoto ", "Esther Kinyanjui", "Muhammad Alnasser", "Rachel Jiang", "Richard Zhu", "Carol Jiang", "Hana Boulware", "Vernon Lin", "Mich

In [5]:
using JuMP
using Gurobi

# Create a new model
model = Model(Gurobi.Optimizer)

num_students = size(df, 1)
num_times = 8
num_classes = 26

# Decision variables
vars = Dict{Tuple{Int, Int, Int, Int}, VariableRef}()

vars_to_minimize_availability_slack = []

for i in 1:num_students
    for j in 1:num_times
        if ismissing(df[i, j]) || df[i, j] == 0  # Handle missing or unavailable volunteer
            # If volunteer not available
            for k in 1:num_classes
                # Decision variables for p & np roles
                vars[(i, j, k, 0)] = @variable(model, binary = true)
                vars[(i, j, k, 1)] = @variable(model, binary = true)

                # Slack variables for unmet availability
                slack_1 = @variable(model)
                push!(vars_to_minimize_availability_slack, slack_1)
                @constraint(model, vars[(i, j, k, 1)] == 0)

                slack_0 = @variable(model)
                push!(vars_to_minimize_availability_slack, slack_0)
                @constraint(model, vars[(i, j, k, 0)] == 0)
            end
            continue
        end

        for k in 1:num_classes
            # Decision variables for available students
            vars[(i, j, k, 0)] = @variable(model, binary = true)
            vars[(i, j, k, 1)] = @variable(model, binary = true)

            # Principal or non-principal roles constraint
            @constraint(
                model,
                vars[(i, j, k, 0)] + vars[(i, j, k, 1)] <= 1,
            )

            # Non-head volunteers cannot be principal
            if head_volunteer[i] != 1
                @constraint(
                    model,
                    vars[(i, j, k, 1)] == 0,
                )
            end
        end
    end
end

# A person can only volunteer for one class at a time
for i in 1:num_students
    for j in 1:num_times
        @constraint(
            model,
            sum(vars[(i, j, k, p)] for k in 1:num_classes for p in 0:1) <= 1,
        )
    end
end

# Variables for slack constraints
vars_to_minimize_slack_p = []
vars_to_minimize_slack = []

# Demand constraints
for j in 1:num_times
    for k in 1:num_classes
        # Slack variable for unmet total demand
        slack = @variable(model)
        push!(vars_to_minimize_slack, slack)

        # Ensure total volunteer demand is met
        @constraint(
            model,
            sum(vars[(i, j, k, p)] for i in 1:num_students for p in 0:1) >= class_data.needed_volunteers[k] - slack,
        )

        # Ensure one principal volunteer per event
        slack_p = @variable(model)
        push!(vars_to_minimize_slack_p, slack_p)

        @constraint(
            model,
            sum(vars[(i, j, k, 1)] for i in 1:num_students) + slack_p == 1,
        )
    end
end

# Variables and constraints for continuity (penalizing discontinuities)
vars_to_minimize = []
vars_to_minimize_constr = []

for i in 1:num_students
    for j in 1:(num_times - 1)
        for k in 1:num_classes
            for p in 0:1
                diff = @variable(model)
                push!(vars_to_minimize, diff)

                # Absolute value constraints
                @constraint(model, diff >= vars[(i, j, k, p)] - vars[(i, j + 1, k, p)])
                @constraint(model, diff >= -(vars[(i, j, k, p)] - vars[(i, j + 1, k, p)]))
            end
        end
    end
end

# Penalize assignments to non-preferred events
vars_to_minimize_pref = []
for i in 1:num_students
    student_pref = s_pref[i]
    for j in 1:num_times
        for k in 1:num_classes
            if k in student_pref
                continue
            end
            for p in 0:1
                push!(vars_to_minimize_pref, vars[(i, j, k, p)])
            end
        end
    end
end

# Set Objectives with priorities
@objective(
    model, Min,
    5 * sum(vars_to_minimize_slack) / length(vars_to_minimize_slack) +
    1 * sum(vars_to_minimize_slack_p) / length(vars_to_minimize_slack_p) +
    1 * sum(vars_to_minimize_pref) / length(vars_to_minimize_pref) +
    1 * sum(vars_to_minimize) / length(vars_to_minimize)
)

# Optimize the model
optimize!(model)

# Check if an optimal solution was found
if termination_status(model) == MOI.OPTIMAL
    println("Optimal solution found.")
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found.")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1250U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 103280 rows, 90376 columns and 342212 nonzeros
Model fingerprint: 0x37023b77
Variable types: 52104 continuous, 38272 integer (38272 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-05, 2e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 7e+00]
Found heuristic solution: objective 14.0769231
Presolve removed 102544 rows and 89596 columns
Presolve time: 0.63s
Presolved: 736 rows, 780 columns, 2600 nonzeros
Found heuristic solution: objective 4.2358510
Variable types: 0 continuous, 780 integer (780 binary)

Root relaxation: objective 4.225932e+00, 0 iterations, 0.00 seconds (0.00 work 

## Output Analysis

In [7]:
using DataFrames
using CSV

# Define timeslots as a vector
timeslots = ["6-8 PM FRI", "8-10 PM FRI", "8-10 AM SAT", "10 AM-12 PM SAT", 
             "12-2 PM SAT", "2-4 PM SAT", "4-6 PM SAT", "6-8 PM SAT"]

# Ensure name_vals and ind_to_name are defined before this point
# Example:
# name_vals = ["Name1", "Name2", ...]
# ind_to_name = Dict(1 => "Name1", 2 => "Name2", ...)

# Initialize a DataFrame to store the results with name column
results_df = DataFrame(names = name_vals)

# Add timeslot columns and initialize with missing values of type String
for slot in timeslots
    results_df[!, slot] = fill("", length(name_vals))  # Initialize with a String
end

# Initialize sumX to accumulate the sum of variable values
sumX = 0.0

# Iterate through the variables in the JuMP model (example)
for (key, var) in vars
    if value(var) > 0
        sumX += value(var)

        name, time, event, principal = key

        # Populate the DataFrame with the corresponding information
        row_index = findfirst(x -> x == ind_to_name[name], results_df.names)
        if !isnothing(row_index)
            results_df[row_index, timeslots[time]] = ind_to_event[event]
        end
    end
end

# Print the first few rows of the DataFrame
println(first(results_df, 5))

# Save the DataFrame to a CSV file
CSV.write("weighted_results.csv", results_df)


5×9 DataFrame
 Row │ names           6-8 PM FRI              8-10 PM FRI             8-10 AM SAT             10 AM-12 PM SAT         12-2 PM SAT             2-4 PM SAT              4-6 PM SAT              6-8 PM SAT             
     │ String31        String                  String                  String                  String                  String                  String                  String                  String                 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Ivan Ge                                                         Optics                  Optics                  Optics                  Optics                  Optics                  Optics
   2 │ Daniel Ochoa                            Anatomy and Physiology  Anatomy and Physiology  Anatomy and Physiology
   3 │ Alan Choi       An

"weighted_results.csv"

In [18]:
sumVals = 0
for var in vars_to_minimize
    sumVals += value(var)
end
println("Sum of vars_to_minimize_pref: ", sumVals)

Sum of vars_to_minimize_pref: 160.0


In [ ]:
print(len(vars_to_minimize_slack))

208


In [ ]:
print(vars_to_minimize_slack[-1].X
      )

0.0
